In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

## 4: Multiplying Matrices
- test that $\sigma_j\sigma_k = \delta_{jk}\mathbf{1} + i \epsilon_{jkl}\sigma_l$

In [2]:
#from 2024-1
def norm_diff(L, M):
    LminusM = L - M
    mag_LminusM = np.sqrt(np.sum(np.abs(LminusM)**2))
    mag_L = np.sqrt(np.sum(np.abs(L)**2))
    mag_M = np.sqrt(np.sum(np.abs(M)**2))
    
    delta = mag_LminusM / (mag_L + mag_M)
    
    return delta

## 5: Create function to multiply matrices
- (5a): MAKE GENERAL MATRIX MAKER: create function that inputs 2 arrays of complex numbers ($A_0, \vec{A}$) and ($B_0, \vec{B}$) and output array of complex numbers: ($C_0, \vec{C}$)
- (5b) create function that outputs array of 4 random complex numbers
- (5c) Test function using random A and B arrays compare two ways

In [3]:
##5a: make general matrix maker
def matrix_maker(m0xyz): #array to matrix
    M0, Mx, My, Mz = m0xyz
    
    identity_matrix = np.zeros((2,2), dtype = 'complex128')
    identity_matrix[0,0]= 1
    identity_matrix[1,1] = 1

    pauli_x = np.zeros((2,2), dtype = 'complex128')
    pauli_x[0,1]= 1
    pauli_x[1,0] = 1

    pauli_y = np.zeros((2,2), dtype = 'complex128')
    pauli_y[0,1]= -1j
    pauli_y[1,0] = 1j

    pauli_z = np.zeros((2,2), dtype = 'complex128')
    pauli_z[0,0]= 1
    pauli_z[1,1] = -1
    
    m = np.array([Mx, My, Mz])
    
    pauli_basis = np.array([pauli_x, pauli_y, pauli_z])
    
    m_dot_pauli = np.tensordot(m, pauli_basis, axes=1)
    
    matrix = M0 * identity_matrix + m_dot_pauli
    
    return matrix

In [4]:
a0xyz = [1, 1, 1, 1]
b0xyz = [2, 2, 2, 2]
A = matrix_maker(a0xyz)
B = matrix_maker(b0xyz)
C = A@B
print(A)
print(B)
print(C)

[[2.+0.j 1.-1.j]
 [1.+1.j 0.+0.j]]
[[4.+0.j 2.-2.j]
 [2.+2.j 0.+0.j]]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [5]:
## takes two arrays and creates a third that can then be used to make a matrix
def calculate_c0xyz(a0xyz, b0xyz): #arrays to array
    A0, Ax, Ay, Az = a0xyz
    B0, Bx, By, Bz = b0xyz
    axyz = np.array([Ax, Ay, Az])
    bxyz = np.array([Bx, By, Bz])
    C0 = A0*B0 + np.tensordot(axyz, bxyz, axes=1)
    cxyz = A0*bxyz + B0*axyz +1j*(np.cross(axyz, bxyz))
    c0xyz = ([C0, cxyz[0], cxyz[1], cxyz[2]])
    return c0xyz

In [6]:
#testing calculate_c0xyz
e0xyz = [1, 1, 1, 1]
f0xyz = [2, 2, 2, 2]
g0xyz = calculate_c0xyz(e0xyz, f0xyz)
print(g0xyz)
G = matrix_maker(g0xyz)
print(G)

[8, (4+0j), (4+0j), (4+0j)]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [7]:
##5b make random number generator
    ## np.random.randn(n) returns a 1D array with n random numbers, so here it makes a 1D array of 4 random numbers (could make a matrix of random numbers or a 2D array i should say, if you give two inputs like np.random.randn(m,n))
    ## and "randn" refers to "random normal" which means it generates numbers from the standard normal distribution (gaussian distribution)
def random_complex_numbers():
    real_parts = np.random.randn(4)
    imaginary_parts = np.random.randn(4)
    complex_numbers = real_parts + 1j*imaginary_parts
    return complex_numbers

random_numbers = random_complex_numbers()
print(random_numbers)

[-0.5996157 -0.28852072j -0.22035786-1.08807166j  0.28342495+0.389017j
  1.02755969+1.68341099j]


In [8]:
#take a0xyz and b0xyz and make c0xyz then C
def one_matrix_test(a0xyz, b0xyz): #arrays to matrix
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    C = matrix_maker(c0xyz)
    return C

#take a0xyz and b0xyz and make A and B then C
def three_matrix_test(a0xyz, b0xyz): #arrays to matrix
    A = matrix_maker(a0xyz)
    B = matrix_maker(b0xyz)
    C = A@B
    return C

In [9]:
#testing
h0xyz = [1, 1, 1, 1]
i0xyz = [2, 2, 2, 2]
C1 = one_matrix_test(h0xyz, i0xyz)
C2 = three_matrix_test(h0xyz, i0xyz)

In [10]:
norm_diff(C1, C2)
print(C1)
print (C2)

[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [11]:
### 5c: testing function using random numbers
def trial_loop(N): 

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()

        #compute density matrix both ways
        L = one_matrix_test(random_numbers1, random_numbers2)
        M = three_matrix_test(random_numbers1, random_numbers2)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2)) #called it something different because i was trying to append random numbers to itslef and it said no
    
    return delta_values, random_numbers_used

In [12]:
N = 100
delta_values, random_numbers_used = trial_loop(N) #run like this to capture outputs

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [6.91301116251582e-17, 5.905973193174746e-17, 2.4432934615688018e-17, 9.83513309285676e-17, 1.160488779509969e-16, 7.472216306057903e-17, 1.0211547850200566e-16, 1.1747509647401452e-16, 9.092133130993408e-17, 9.993275354170885e-17, 5.935041594093014e-17, 8.12031687477093e-17, 6.578618620138111e-17, 5.554027981818899e-17, 1.1449168167017715e-16, 2.415417334977992e-17, 1.1774297912207793e-16, 7.697674689491446e-17, 6.776320693022863e-17, 1.3901312616755555e-16, 1.0804999877682817e-16, 6.905953231573605e-17, 6.429890225930654e-17, 9.758361136967257e-17, 7.710718860950003e-17, 3.3176116036102085e-17, 1.1406695064131582e-16, 8.297703752060837e-17, 9.820148569219525e-17, 9.340706698862317e-17, 7.79096435455931e-17, 6.371518946814279e-17, 2.743065923837112e-17, 7.183894262715731e-17, 8.584489333469504e-17, 7.713462001932333e-17, 7.836189641399514e-17, 3.754140366675993e-17, 4.033951937121068e-17, 8.61051275472353e-17, 8.856638966879808e-17, 8.24826271141716e-17, 9.01281032493924

## 6: Create matrix multiplication for $(\mathbf{1} - \rho^{(2)})\rho^{(4)}$
- (6a): create functions that input $(P_0, \vec{P})$ and output $(A_0, \vec{A})$ and $(B_0, \vec{B})$
- (6b): test

In [13]:
## 6a
def array_for_rho4(p0xyz): 
    P0, Px, Py, Pz = p0xyz
    pxyz = np.array([Px, Py, Pz])
    B0 = 0.5*P0
    bxyz = 0.5*P0*pxyz
    b0xyz = ([B0, bxyz[0], bxyz[1], bxyz[2]])
    return b0xyz #makes array to be put into matrix maker to create rho

def array_for_rho2(p0xyz):
    P0, Px, Py, Pz = p0xyz
    pxyz = np.array([Px, Py, Pz])
    A0 = 1 - 0.5*P0
    axyz = -0.5*P0*pxyz
    a0xyz = ([A0, axyz[0], axyz[1], axyz[2]])
    return a0xyz
#i think the point of this is to take the specific array and matrix and make it general so I can put into general matrix maker. If i was using create_density_matrix, i could probably put p0xyz directly in and it would make the matrix?

^ i think the point of this is to take the specific array and matrix and make it general so I can put into general matrix maker. If i was using create_density_matrix, i could probably put p0xyz directly in and it would make the matrix?

In [14]:
## 6b testing
p0xyz = [1, 1, 1, 1]
b0xyz = array_for_rho4(p0xyz) #how to check this
print('ARRAY B:', b0xyz)
rho4 = matrix_maker(b0xyz)
print('RHO4:',rho4)

p0xyz = [1, 1, 1, 1]
a0xyz = array_for_rho2(p0xyz) #how to check this
print('ARRAY A:', a0xyz)
rho2 = matrix_maker(a0xyz)
print('RHO2:', rho2)

ARRAY B: [0.5, 0.5, 0.5, 0.5]
RHO4: [[1. +0.j  0.5-0.5j]
 [0.5+0.5j 0. +0.j ]]
ARRAY A: [0.5, -0.5, -0.5, -0.5]
RHO2: [[ 0. +0.j  -0.5+0.5j]
 [-0.5-0.5j  1. +0.j ]]


## 7: Calculate $ C = (\mathbf{1} - \rho^{(2)})\rho^{(4)} = C_0\mathbf{1} + \vec{C}* \vec{\sigma}$
- (7a): create function that inputs 2 arrays ($P_0^{(2)}, \vec{P^{(2)}}$) and ($P_0^{(4)}, \vec{P^{(4)}}$) and outputs $(C_0, \vec{C})$
- (7b): test

- $\textbf{create_rho2}$ creates an array that can be put into matrix maker to create matrix "rho2"
- $\textbf{create_rho4}$ creates an array that can be put into matrix maker to create matrix "rho4"

- "rho2" is a matrix:  $(\mathbf{1} - \rho^{(2)})$
- "rho4" is a matrix: $\rho^{(4)}$

- so to create rho2 or rho4, need to take p2 or p4 and input it into the correct function, then we will have an array to put into matrix maker to get rho2 and rho4?
..... i am getting lost

In [15]:
#### dont need this: do need this!!!
def rho2_matrix(p0xyz_2):
    rho2_array = array_for_rho2(p0xyz_2)
    rho2 = matrix_maker(rho2_array)
    return rho2

def rho4_matrix(p0xyz_4):
    rho4_array = array_for_rho4(p0xyz_4)
    rho4 = matrix_maker(rho4_array)
    return rho4

In [16]:
##7a
def rho2timesrho4(p0xyz_2, p0xyz_4):
    #make a0xyz and b0xyz from p0xyz_2 and p0xyz_4 -- thats what array_for_rho4 functions do!
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    return c0xyz

p0xyz_2 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]
c10xyz = rho2timesrho4(p0xyz_2, p0xyz_4)
C1 = matrix_maker(c10xyz)
print(c10xyz)
print(C1)

[-0.5, 0j, 0j, 0j]
[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


COMPARING FUNCTIONS:
- function 1, $\textbf{final_C_1}$ is taking in p2 and p4 arrays, creating a0xyz and b0xyz arrays to then get a third array c0xyz which can go into matrix maker and get the final matrix C
- function 2, $\textbf{final_C_2}$ is taking in p2 and p4 arrays and making rho2 and rho4 matrices to then be multiplied and create matrix C


In [17]:
##7b: comparing functions:
%time
def final_C_1(p0xyz_2, p0xyz_4):
    #make a0xyz and b0xyz from p0xyz_2 and p0xyz_4 -- thats what array_for_rho4 functions do!
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    C1 = matrix_maker(c0xyz)
    return C1
%time
def final_C_2(p0xyz_2, p0xyz_4):
    rho2 = rho2_matrix(p0xyz_2)
    rho4 = rho4_matrix(p0xyz_4)
    C2 = rho2@rho4
    return C2

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs
CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 1.91 µs


In [18]:
p0xyz_2 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]
C1 = final_C_1(p0xyz_2, p0xyz_4)
C2 = final_C_2(p0xyz_2, p0xyz_4)
print (C1)
print (C2)

[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]
[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


In [19]:
### testing with random numbers
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        
        L = final_C_1(random_numbers1, random_numbers2)
        M = final_C_2(random_numbers1, random_numbers2)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2)) #called it something different because i was trying to append random numbers to itslef and it said no
    
    return delta_values, random_numbers_used

In [20]:
N = 100 
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [7.59606053194603e-17, 1.0740973255713397e-16, 5.0222787453218554e-17, 6.19435646307528e-17, 6.274983047721381e-17, 7.021489976371722e-17, 9.631944797555075e-17, 6.00653923876215e-17, 8.910081759809231e-17, 6.434389579283101e-17, 4.307304520745248e-17, 9.517619114759434e-17, 3.7879724835029844e-17, 8.885321804497694e-17, 8.898682856464875e-17, 7.88556952145077e-17, 3.3362180668501984e-17, 6.5076832408869e-17, 9.681440054817009e-17, 7.230360559240179e-17, 1.010424498533906e-16, 8.906231004704688e-17, 6.339796363898152e-17, 6.449665630875839e-17, 5.773625529120045e-17, 9.14683521146771e-17, 8.852171382898224e-17, 1.0428928292290923e-16, 6.4657629356412e-17, 7.279838619670556e-17, 5.597262415535771e-17, 8.085956284429371e-17, 6.004544262125069e-17, 8.980394627575709e-17, 7.611947147936777e-17, 6.294617916574725e-17, 7.254242450849401e-17, 1.1044808288625503e-16, 1.0518126280166244e-16, 6.977638838380555e-17, 8.333556313054734e-17, 7.954796331997822e-17, 9.658004911791434e-17

## 8: Traces
- (8a): find $D = C + \mathrm{trc}(C)\mathbf{1} = D_0\mathbf{1} + \vec{D} * \vec{\sigma}$
- (8b): Test the equivalent of $(D_0, \vec{D})$ and $D = C + \mathrm{trc}(C)\mathbf{1}$

In [21]:
##8a
c0xyz = rho2timesrho4(p0xyz_2, p0xyz_4) #arrays were generated in cell 18
print('check C array:', c0xyz)

check C array: [-0.5, 0j, 0j, 0j]


In [22]:
C = final_C_1(p0xyz_2, p0xyz_4) #can use either function c1 or c2 both make the matrix c
print ('check matrix C:', C)

check matrix C: [[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


In [23]:
trC = np.trace(C)
print('trace of C=', trC, '2C_0=', 2*c0xyz[0])

identity_matrix = np.zeros((2,2), dtype = 'complex128')
identity_matrix[0,0]= 1
identity_matrix[1,1] = 1

trace of C= (-1+0j) 2C_0= -1.0


In [24]:
D = C + np.trace(C)*identity_matrix
print('check matrix D:', D)

check matrix D: [[-1.5+0.j  0. +0.j]
 [ 0. +0.j -1.5+0.j]]


In [25]:
#making matrix D another way: takes matrix D and produces d0xyz?
D2 = C + 2*c0xyz[0]
print (D)
print (D2)

[[-1.5+0.j  0. +0.j]
 [ 0. +0.j -1.5+0.j]]
[[-1.5+0.j -1. +0.j]
 [-1. +0.j -1.5+0.j]]


## 9: Creating the real functions!
- create functions to calculate E and F:
- $E = \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big] $
    - $E = \rho^{(3)} \big[C + \mathrm{tr}(C)\big] $
- $F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big] $
    - $F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[C + \mathrm{tr}(C)\big] $

#### THE BIG IDEA:

so the idea will be to slowly build up to function F
- first we create $(\mathbf{1} -\rho^{(2)})\rho^{(4)}$, which we call C
    - call $\mathbf{rho2timesrho4}$(p0xyz_2, p0xyz_4) and it returns c0xyz
    - the you would do matrix C = $\textbf{matrix_maker}$(c0xyz)
    - OR call $\textbf{final_C_1}$(p0xyz_2, p0xyz_4) 
    - ***** use final c1 or final c2?

- then we will create $(\mathbf{1} -\rho^{(2)})\rho^{(4)}+ \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)$, which we will call D (just adding trace * identity onto C)
- then we will multiply matrix $\rho^{(3)}$, which we will call E
- lastly, we will multiply matrix $(\mathbf{1} -\rho^{(1)})$, which is then F

- Input: $P^{(1)}$, $P^{(2)}$, $P^{(3)}$, $P^{(4)}$, which I call p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4
- (10a): Output: $(2\mathrm{Re}[F_0], 2\mathrm{Re}[\vec{F}])$

In [26]:
# define final_C_1 if i change the function name...
# define identity_matrix
# need matrix_maker
# need to define P vectors
p0xyz_1 = [1, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]

In [27]:
## need new name for this
def matrix_mult_arrays(a0xyz, b0xyz): #arrays to array
    A0, Ax, Ay, Az = a0xyz
    B0, Bx, By, Bz = b0xyz
    axyz = np.array([Ax, Ay, Az])
    bxyz = np.array([Bx, By, Bz])
    C0 = A0*B0 + np.tensordot(axyz, bxyz, axes=1)
    cxyz = A0*bxyz + B0*axyz +1j*(np.cross(axyz, bxyz))
    c0xyz = ([C0, cxyz[0], cxyz[1], cxyz[2]])
    return c0xyz

In [28]:
c0xyz =[2, 1, 1, 1]
d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
print (d0xyz)

[6, 1, 1, 1]


In [29]:
## okay so we are going to make 2 functions to calculate F.... 
## one does the RHS which does matrix all the way
## the other does the LHS which keeps arrays all the way

#this function is for testing
def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = final_C_1(p0xyz_2, p0xyz_4)
    #print ('C=', C)
    D = C + np.trace(C)*identity_matrix
    #print ('D=', D)
    E = matrix_maker(p0xyz_3)@D
    #print ('E=', E)
    F = matrix_maker(p0xyz_1)@E
    return (F)

#this function will go into code
def F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    #convert p's to a and b - what function does this? array_for_rho2 and array_for_rho4
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
   
    #multiply a and b
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)

    #add d
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
   
    #multiply p0xyz_3 to get e
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
   
    #multiply p0xyz_1 to get f
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
   
    #make array matrix
    F = matrix_maker(f0xyz)
    return (F)

In [30]:
p0xyz_1 = [1, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]

print ('INPUT P ARRAYS')
print('P1=', p0xyz_1)
print('P2=', p0xyz_2)
print('P3=', p0xyz_3)
print('P4=', p0xyz_4)

print ('CONVERTED ARRAYS')
print('a0xyz=', array_for_rho2(p0xyz_2))
print('b0xyz=', array_for_rho4(p0xyz_4))
print('c0xyz=', matrix_mult_arrays(a0xyz, b0xyz))

print ('MATRICES')
print ('A=',rho2_matrix(p0xyz_2))
print ('B=', rho4_matrix(p0xyz_4))

print ('F made with matrices', F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4))

print ('F made with arrays', F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4))

# compare both ways
FM = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
FA = F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print(type(FA))
print(type(FM))

INPUT P ARRAYS
P1= [1, 1, 1, 1]
P2= [1, 1, 1, 1]
P3= [1, 1, 1, 1]
P4= [1, 1, 1, 1]
CONVERTED ARRAYS
a0xyz= [0.5, -0.5, -0.5, -0.5]
b0xyz= [0.5, 0.5, 0.5, 0.5]
c0xyz= [-0.5, 0j, 0j, 0j]
MATRICES
A= [[ 0. +0.j  -0.5+0.5j]
 [-0.5-0.5j  1. +0.j ]]
B= [[1. +0.j  0.5-0.5j]
 [0.5+0.5j 0. +0.j ]]
F made with matrices [[-9.+0.j -3.+3.j]
 [-3.-3.j -3.+0.j]]
F made with arrays [[-9.+0.j -3.+3.j]
 [-3.-3.j -3.+0.j]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [31]:
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways
        L = F_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = F_array_to_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [32]:
N = 100
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [1.9984235671033094e-16, 1.3624570128329183e-16, 7.658969608149081e-17, 1.8277496389426e-16, 7.057947214607825e-17, 1.8668065532290156e-16, 8.676925814869243e-17, 1.449426236084136e-16, 1.024259284675586e-16, 1.2571973902467764e-16, 1.26980859937626e-16, 8.806841758877319e-17, 1.2910160521028532e-16, 1.3859001690902095e-16, 1.2321358315818765e-16, 1.7158320161287706e-16, 1.107857853016067e-16, 3.995769785368358e-16, 6.048669111260601e-17, 1.1941693389150627e-16, 1.4664597720453862e-16, 1.523104156755515e-16, 1.450288387693679e-16, 8.407414640603387e-17, 1.2889246884321743e-16, 8.475269202269607e-17, 8.312935157525885e-17, 8.245302295611029e-17, 5.864613643809893e-17, 1.428696858708901e-16, 8.705126523622644e-17, 8.352617744037682e-17, 7.137363715668625e-17, 2.2477155952186856e-16, 9.106925108101651e-17, 1.250549781483327e-16, 9.076307179501511e-17, 7.804605920333678e-17, 1.6023235562907325e-16, 1.1140389068922635e-16, 1.4329560177760894e-16, 1.1304941752988463e-16, 1.4324

## 10: Function from Bennett et al
- make function that inputs the 4 p arrays and outputs an array of REAL NUMBERS so output: $(2\mathrm{Re}[F_0], 2\mathrm{Re}[\vec{F}])$

testing complex conjugate stuff:

In [33]:
# found np.conjugate(): function that takes the complex conjugate
z = 3 + 2*1j
z_conj = np.conjugate(z)
print(z, z_conj)

(3+2j) (3-2j)


In [34]:
z = [1, -1j, 1j, 1]
z_conj = np.conjugate(z)
print(z, z_conj)

[1, (-0-1j), 1j, 1] [ 1.-0.j -0.+1.j  0.-1.j  1.-0.j]


In [35]:
def F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    return (f0xyz)

def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = final_C_1(p0xyz_2, p0xyz_4)
    D = C + np.trace(C)*identity_matrix
    E = matrix_maker(p0xyz_3)@D
    F = matrix_maker(p0xyz_1)@E
    return (F)

p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]

f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
f0xyz_conj = np.conjugate(f0xyz)

print('ARRAY STUFF')
print('f0xyz=',f0xyz)
print('f0xyz_conj=',f0xyz_conj)
Re_f0xyz = f0xyz + f0xyz_conj
print('Re_f0xyz', Re_f0xyz)

Re_F_made_w_arrays = matrix_maker(Re_f0xyz)
print(Re_F_made_w_arrays)

F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
F_conj = np.conjugate(F)

print('MATRIX STUFF')
print('F=',F)
print('F_conj=',F_conj)
Re_F = F + F_conj
print('Re_F=',Re_F)

ARRAY STUFF
f0xyz= [(-3+1j), (-2+0j), (-1-1j), (-2+0j)]
f0xyz_conj= [-3.-1.j -2.-0.j -1.+1.j -2.-0.j]
Re_f0xyz [-6.+0.j -4.+0.j -2.+0.j -4.+0.j]
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]
MATRIX STUFF
F= [[-5.+1.j -3.+1.j]
 [-1.-1.j -1.+1.j]]
F_conj= [[-5.-1.j -3.-1.j]
 [-1.+1.j -1.-1.j]]
Re_F= [[-10.+0.j  -6.+0.j]
 [ -2.+0.j  -2.+0.j]]


In [52]:
## 10a:
# the funciton as is outputs the F matrix, we want to take that and
# take the real part and print out the array of that
def Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

def arr_2_mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

In [60]:
p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]


Re_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
Fvvsc_from_array = matrix_maker(Re_f0xyz)
print('Fvvsc_array PRODUCES:')
print(Fvvsc_from_array)

Fvvsc_arr_2_mat = Fvvsc_arr_2_mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print('Fvvsc_arr_2_mat PRODUCES:')
#print(Fvvsc_arr_2_mat)

F_made_with_matrices = Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print('Fvvsc_matrix PRODUCES:')
print(F_made_with_matrices)

Fvvsc_array PRODUCES:
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]


TypeError: 'numpy.ndarray' object is not callable

In [56]:
## testing
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways   
        L = Fvvsc_arr_2_mat(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = Fvvsc_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [57]:
N = 10
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

TypeError: 'numpy.ndarray' object is not callable